In [1]:
import pandas as pd
from pprint import pprint
# from config import api_key
import requests
import time

### Extract and clean college data from csv

In [2]:
# Read the colleges data from the csv file
college_csv = pd.read_csv("Resources/Colleges_and_Universities.csv")
college_csv.head()

,X,Y,OBJECTID,IPEDSID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,ALIAS,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID
0,-8.765605e+06,5.318192e+06,1,189556,BRYANT & STRATTON COLLEGE-AMHERST,3650 MILLERSPORT HIGHWAY,GETZVILLE,NY,14068,NOT AVAILABLE,...,NOT AVAILABLE,6,1,141,176,317,2,-999,79,NOT AVAILABLE
1,-8.231383e+06,4.975222e+06,2,190682,CUNY SCHOOL OF LAW,2 COURT SQUARE,LONG ISLAND CITY,NY,11101,NOT AVAILABLE,...,LAW SCHOOL | CUNY LAW SCHOOL,18,1,165,418,583,2,-999,201,NOT AVAILABLE
2,-8.247285e+06,5.032103e+06,3,195058,ROCKLAND COMMUNITY COLLEGE,145 COLLEGE RD,SUFFERN,NY,10901,3699,...,NOT AVAILABLE,3,3,3317,3542,6859,2,-999,901,NOT AVAILABLE
3,-8.208890e+06,5.008471e+06,4,193122,SOUTHERN WESTCHESTER BOCES-PRACTICAL NURSING P...,450 MAMARONECK AVENUE,HARRISON,NY,10528,NOT AVAILABLE,...,NOT AVAILABLE,-2,1,54,52,106,2,-999,31,NOT AVAILABLE
4,-8.969252e+06,4.965197e+06,5,204884,OHIO VALLEY COLLEGE OF TECHNOLOGY,15258 STATE ROUTE 170,EAST LIVERPOOL,OH,43920,9585,...,NOT AVAILABLE,1,1,25,176,201,2,-999,30,NOT AVAILABLE


In [3]:
# Select the relevant columns from the dataframe
college_data = college_csv[['IPEDSID', 'NAME', 'ADDRESS','CITY', 'COUNTY', 'STATE', 'ZIP', 'COUNTRY', 'TELEPHONE', \
        'WEBSITE', 'LATITUDE', 'LONGITUDE']]
# Rename the columns
college_data.columns=('College_Id', 'College_Name', 'Address', 'City', 'County', 'State', 'Zip', 'Country', 'Telephone', \
                            'Website', 'Latitude', 'Longitude')
college_data.reset_index(inplace=True, drop=True)
college_data.head()

,College_Id,College_Name,Address,City,County,State,Zip,Country,Telephone,Website,Latitude,Longitude
0,189556,BRYANT & STRATTON COLLEGE-AMHERST,3650 MILLERSPORT HIGHWAY,GETZVILLE,ERIE,NY,14068,USA,(716) 625-6300,https://www.bryantstratton.edu,43.040850,-78.742765
1,190682,CUNY SCHOOL OF LAW,2 COURT SQUARE,LONG ISLAND CITY,QUEENS,NY,11101,USA,(718) 340-4200,www.law.cuny.edu,40.747863,-73.943769
2,195058,ROCKLAND COMMUNITY COLLEGE,145 COLLEGE RD,SUFFERN,ROCKLAND,NY,10901,USA,(845) 574-4000,www.sunyrockland.edu/,41.133841,-74.086622
3,193122,SOUTHERN WESTCHESTER BOCES-PRACTICAL NURSING P...,450 MAMARONECK AVENUE,HARRISON,WESTCHESTER,NY,10528,USA,(914) 592-0849,www.swboces.org,40.973755,-73.741713
4,204884,OHIO VALLEY COLLEGE OF TECHNOLOGY,15258 STATE ROUTE 170,EAST LIVERPOOL,COLUMBIANA,OH,43920,USA,(330) 385-1070,ovct.edu,40.679602,-80.572164


##### Cleaned College master data

In [4]:
college_data.head(3)

,College_Id,College_Name,Address,City,County,State,Zip,Country,Telephone,Website,Latitude,Longitude
0,189556,BRYANT & STRATTON COLLEGE-AMHERST,3650 MILLERSPORT HIGHWAY,GETZVILLE,ERIE,NY,14068,USA,(716) 625-6300,https://www.bryantstratton.edu,43.040850,-78.742765
1,190682,CUNY SCHOOL OF LAW,2 COURT SQUARE,LONG ISLAND CITY,QUEENS,NY,11101,USA,(718) 340-4200,www.law.cuny.edu,40.747863,-73.943769
2,195058,ROCKLAND COMMUNITY COLLEGE,145 COLLEGE RD,SUFFERN,ROCKLAND,NY,10901,USA,(845) 574-4000,www.sunyrockland.edu/,41.133841,-74.086622


### Extract and transform data from JSon File

In [5]:
from config import api_key

base_url="https://api.data.gov/ed/collegescorecard/v1/schools.json?school.degrees_awarded.predominant=2,3"
page_no = 0
perpage = 100

fields=f"&fields=id,school.name,school.main_campus,school.branches,school.price_calculator_url,latest.student.size,latest.admissions,latest.academics.program.degree,latest.cost"
sub_query = f"{fields}&per_page={perpage}&api_key="
query_url = f"{base_url}{sub_query}{api_key}"

In [6]:
def getPrograms(value):
    programs = []
    
    if (value['latest.academics.program.degree.agriculture']):
        programs.append('Agriculture')
    if (value['latest.academics.program.degree.architecture']):
        programs.append('Architecture')
    if (value['latest.academics.program.degree.biological']):
        programs.append('Biological')
    if (value['latest.academics.program.degree.business_marketing']):
        programs.append('Business Marketing')
    if (value['latest.academics.program.degree.communication']):
        programs.append('Communication')
    if (value['latest.academics.program.degree.communications_technology']):
        programs.append('Communications Technology')
    if (value['latest.academics.program.degree.computer']):
        programs.append('Computer')
    if (value['latest.academics.program.degree.construction']):
        programs.append('Construction')        
    if (value['latest.academics.program.degree.education']):
        programs.append('Education')
    if (value['latest.academics.program.degree.engineering']):
        programs.append('Engineering')
    if (value['latest.academics.program.degree.engineering_technology']):
        programs.append('Engineering Technology')
    if (value['latest.academics.program.degree.english']):
        programs.append('English')
    if (value['latest.academics.program.degree.ethnic_cultural_gender']):
        programs.append('Ethnic Cultural Gender')
    if (value['latest.academics.program.degree.family_consumer_science']):
        programs.append('Family Consumer Science')
    if (value['latest.academics.program.degree.health']):
        programs.append('Health')
    if (value['latest.academics.program.degree.history']):
        programs.append('History')
    if (value['latest.academics.program.degree.humanities']):
        programs.append('Humanities')
    if (value['latest.academics.program.degree.language']):
        programs.append('Language')
    if (value['latest.academics.program.degree.legal']):
        programs.append('Legal')
    if (value['latest.academics.program.degree.library']):
        programs.append('Library')
    if (value['latest.academics.program.degree.mathematics']):
        programs.append('Mathematics')
    if (value['latest.academics.program.degree.mechanic_repair_technology']):
        programs.append('Mechanic Repair Technology')
    if (value['latest.academics.program.degree.military']):
        programs.append('Military')
    if (value['latest.academics.program.degree.multidiscipline']):
        programs.append('Multidiscipline')
    if (value['latest.academics.program.degree.parks_recreation_fitness']):
        programs.append('Parks Recreation Fitness')
    if (value['latest.academics.program.degree.personal_culinary']):
        programs.append('Personal Culinary')
    if (value['latest.academics.program.degree.philosophy_religious']):
        programs.append('Philosophy Religious')
    if (value['latest.academics.program.degree.physical_science']):
        programs.append('Physical Science')
    if (value['latest.academics.program.degree.precision_production']):
        programs.append('Precision Production')
    if (value['latest.academics.program.degree.psychology']):
        programs.append('Psychology')
    if (value['latest.academics.program.degree.public_administration_social_service']):
        programs.append('Public Administration Social Service')
    if (value['latest.academics.program.degree.resources']):
        programs.append('Resources')
    if (value['latest.academics.program.degree.science_technology']):
        programs.append('Science Technology')
    if (value['latest.academics.program.degree.security_law_enforcement']):
        programs.append('Security Law Enforcement')
    if (value['latest.academics.program.degree.social_science']):
        programs.append('Social Science')
    if (value['latest.academics.program.degree.theology_religious_vocation']):
        programs.append('Theology Religious Vocation')
    if (value['latest.academics.program.degree.transportation']):
        programs.append('Transportation')
    if (value['latest.academics.program.degree.visual_performing']):
        programs.append('Visual Performing')

    return str(programs).strip('[]')

In [7]:
# # Write the API response data to a json file
# import json
# with open('mydata-2.json', 'w') as f:
#     json.dump(pprint(response), f)

In [8]:
pageno = 0
query_url = f"{query_url}&page={pageno}"
get_response = requests.get(query_url)
response = get_response.json()

In [9]:
tot_records = response['metadata']['total']
no_of_pages = int(tot_records/100)
print(no_of_pages)

32


In [10]:
dictionary_list = []
programs_list = []

pageno = 0
for page in range(no_of_pages):
    query_url = f"{query_url}&page={pageno}"
    get_response = requests.get(query_url)
    response = get_response.json()
    results_set = response['results']
    print(f"........Page {page}.....")
    for val in results_set:
        col_dict = {}
        prog_list = getPrograms(val)
        col_dict = {'College_Id':val['id'],
                    'price_calc_url': val['school.price_calculator_url'],
                    'main_campus': val['school.main_campus'],
                    'branches': val['school.branches'],
                    'student_size' : val['latest.student.size'],   
                    'programs': prog_list,
                    'act_writing': val['latest.admissions.act_scores.75th_percentile.writing'],
                    'act_math' : val['latest.admissions.act_scores.75th_percentile.math'],
                    'act_eng': val['latest.admissions.act_scores.75th_percentile.english'],
                    'act_net': val['latest.admissions.act_scores.75th_percentile.cumulative'],
                    'sat_writing' : val['latest.admissions.sat_scores.75th_percentile.writing'],
                    'sat_reading' : val['latest.admissions.sat_scores.75th_percentile.critical_reading'],
                    'sat_math' : val['latest.admissions.sat_scores.75th_percentile.math'],
                    'sat_avg' : val['latest.admissions.sat_scores.average.overall'],
                    'tution_instate': val['latest.cost.tuition.in_state'],
                    'tution_out_of_state' : val['latest.cost.tuition.out_of_state'],
                    'net_cost_avg' : val['latest.cost.avg_net_price.overall'],
                    'net_cost_attend': val['latest.cost.attendance.academic_year'],
                    'net_cost_private': val['latest.cost.avg_net_price.private'],
                    'cost_inc_public' : val['latest.cost.net_price.public.by_income_level.75000-plus'],
                    'cost_inc_private': val['latest.cost.net_price.private.by_income_level.75000-plus'],
                    'cost_inc_avg' : val['latest.cost.net_price.consumer.by_income_level.75000-plus']
                   }
        dictionary_list.append(col_dict)
        
pprint(dictionary_list)
print("...........................................")
print("         Data Extraction Completed         ")
print("...........................................")

........Page 0.....
........Page 1.....
........Page 2.....
........Page 3.....
........Page 4.....
........Page 5.....
........Page 6.....
........Page 7.....
........Page 8.....
........Page 9.....
........Page 10.....
........Page 11.....
........Page 12.....
........Page 13.....
........Page 14.....
........Page 15.....
........Page 16.....
........Page 17.....
........Page 18.....
........Page 19.....
........Page 20.....
........Page 21.....
........Page 22.....
........Page 23.....
........Page 24.....
........Page 25.....
........Page 26.....
........Page 27.....
........Page 28.....
........Page 29.....
........Page 30.....
........Page 31.....
[{'College_Id': 194028,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 15937,
  'cost_inc_private': None,
  'cost_inc_public': 15937,
  'main_campus': 1,
  'net_cost_attend': 16374,
  'net_cost_avg': 10751,
  'net_cost_private': None,
  'price_calc_url': 'www.suny.edu/

  'cost_inc_avg': 14549,
  'cost_inc_private': None,
  'cost_inc_public': 14549,
  'main_campus': 1,
  'net_cost_attend': 15049,
  'net_cost_avg': 11886,
  'net_cost_private': None,
  'price_calc_url': 'www.hawkeyecollege.edu/financial-aid/net-price-calculator.aspx',
  'programs': "'Agriculture', 'Business Marketing', 'Communication', "
              "'Communications Technology', 'Computer', 'Construction', "
              "'Engineering Technology', 'Family Consumer Science', 'Health', "
              "'Humanities', 'Mechanic Repair Technology', 'Multidiscipline', "
              "'Parks Recreation Fitness', 'Precision Production', "
              "'Resources', 'Security Law Enforcement', 'Visual Performing'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 3217,
  'tution_instate': 5311,
  'tution_out_of_state': 6021},
 {'College_Id': 154998,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branc

  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 14338,
  'cost_inc_private': None,
  'cost_inc_public': 14338,
  'main_campus': 1,
  'net_cost_attend': 14983,
  'net_cost_avg': 10418,
  'net_cost_private': None,
  'price_calc_url': 'https://www.emcc.edu/wp-content/uploads/2017/12/npcalc-rev.htm',
  'programs': "'Business Marketing', 'Computer', 'Construction', 'Education', "
              "'Engineering Technology', 'Health', 'Humanities', 'Mechanic "
              "Repair Technology', 'Multidiscipline', 'Parks Recreation "
              "Fitness', 'Personal Culinary', 'Precision Production', "
              "'Security Law Enforcement'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 1647,
  'tution_instate': 3779,
  'tution_out_of_state': 6599},
 {'College_Id': 161688,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 14768,
  'cost_inc_private': None,
  '

  'cost_inc_private': None,
  'cost_inc_public': None,
  'main_campus': 1,
  'net_cost_attend': 12565,
  'net_cost_avg': 3628,
  'net_cost_private': 3628,
  'price_calc_url': 'https://uttc.edu/financial-aid/net-price-calculator/',
  'programs': "'Business Marketing', 'Computer', 'Education', 'Engineering', "
              "'Health', 'Humanities', 'Mechanic Repair Technology', 'Parks "
              "Recreation Fitness', 'Personal Culinary', 'Public "
              "Administration Social Service', 'Resources', 'Security Law "
              "Enforcement', 'Visual Performing'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 399,
  'tution_instate': 4252,
  'tution_out_of_state': 4252},
 {'College_Id': 199722,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': None,
  'cost_inc_private': None,
  'cost_inc_public': None,
  'main_campus': 1,
  'net_cost_attend': 10813,
  '

  'tution_out_of_state': 11544},
 {'College_Id': 146418,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 12326,
  'cost_inc_private': None,
  'cost_inc_public': 12326,
  'main_campus': 1,
  'net_cost_attend': 12741,
  'net_cost_avg': 7173,
  'net_cost_private': None,
  'price_calc_url': 'www.kish.edu/admissions-financial/cost-attendance',
  'programs': "'Agriculture', 'Business Marketing', 'Computer', 'Education', "
              "'Engineering', 'Engineering Technology', 'Family Consumer "
              "Science', 'Health', 'Humanities', 'Mechanic Repair Technology', "
              "'Multidiscipline', 'Security Law Enforcement', "
              "'Transportation', 'Visual Performing'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 2295,
  'tution_instate': 4800,
  'tution_out_of_state': 13320},
 {'College_Id': 147378,
  'act_eng': None,
  'act_math': None,
  'act

...

  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 14768,
  'cost_inc_private': None,
  'cost_inc_public': 14768,
  'main_campus': 1,
  'net_cost_attend': 15518,
  'net_cost_avg': 10164,
  'net_cost_private': None,
  'price_calc_url': 'services.allegany.edu/netprice/',
  'programs': "'Business Marketing', 'Communications Technology', 'Computer', "
              "'Education', 'Engineering Technology', 'Health', 'Humanities', "
              "'Legal', 'Mechanic Repair Technology', 'Personal Culinary', "
              "'Resources', 'Security Law Enforcement'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 1833,
  'tution_instate': 4220,
  'tution_out_of_state': 9290},
 {'College_Id': 169992,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 10947,
  'cost_inc_private': None,
  'cost_inc_public': 10947,
  'main_campus': 1,
  'net_cost_attend': 12385,
  'net_cost

 {'College_Id': 205391,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 33563,
  'cost_inc_private': 33563,
  'cost_inc_public': None,
  'main_campus': 1,
  'net_cost_attend': 35783,
  'net_cost_avg': 30346,
  'net_cost_private': 30346,
  'price_calc_url': 'https://themodern.edu/tuition/net-price-calculator/',
  'programs': "'Visual Performing'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 210,
  'tution_instate': 28278,
  'tution_out_of_state': 28278},
 {'College_Id': 205841,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 8326,
  'cost_inc_private': None,
  'cost_inc_public': 8326,
  'main_campus': 1,
  'net_cost_attend': 9232,
  'net_cost_avg': 5157,
  'net_cost_private': None,
  'price_calc_url': 'www.starkstate.edu/admissions/finaid/npcalculator/',
  'programs': "'Biological', 'Business Ma

  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': None,
  'cost_inc_private': None,
  'cost_inc_public': None,
  'main_campus': 1,
  'net_cost_attend': 14171,
  'net_cost_avg': 8553,
  'net_cost_private': None,
  'price_calc_url': 'www.sampsoncc.edu/student-services/net-price-calculator/npcalc.html',
  'programs': "'Agriculture', 'Business Marketing', 'Computer', "
              "'Construction', 'Education', 'Engineering Technology', "
              "'Health', 'Humanities', 'Language', 'Personal Culinary', "
              "'Precision Production', 'Security Law Enforcement'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 1231,
  'tution_instate': 2783,
  'tution_out_of_state': 9695},
 {'College_Id': 198905,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 11043,
  'cost_inc_private': None,
  'cost_in

  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 11059,
  'cost_inc_private': None,
  'cost_inc_public': 11059,
  'main_campus': 1,
  'net_cost_attend': 12448,
  'net_cost_avg': 7580,
  'net_cost_private': None,
  'price_calc_url': 'www.hutchcc.edu/cost-and-fees',
  'programs': "'Agriculture', 'Biological', 'Business Marketing', "
              "'Communications Technology', 'Computer', 'Construction', "
              "'Engineering Technology', 'Family Consumer Science', 'Health', "
              "'Humanities', 'Legal', 'Mechanic Repair Technology', 'Personal "
              "Culinary', 'Precision Production', 'Security Law Enforcement'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 3442,
  'tution_instate': 3200,
  'tution_out_of_state': 4512},
 {'College_Id': 160995,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 16214,
  'cost_inc_private': 16214,
  

 {'College_Id': 188058,
  'act_eng': None,
  'act_math': None,
  'act_net': None,
  'act_writing': None,
  'branches': 1,
  'cost_inc_avg': 10508,
  'cost_inc_private': None,
  'cost_inc_public': 10508,
  'main_campus': 1,
  'net_cost_attend': 13230,
  'net_cost_avg': 8224,
  'net_cost_private': None,
  'price_calc_url': 'nnmc.edu/wp-content/uploads/2016/09/NetPriceCalculator16/npcalc.htm',
  'programs': "'Biological', 'Business Marketing', 'Computer', 'Construction', "
              "'Education', 'Engineering', 'Engineering Technology', 'Ethnic "
              "Cultural Gender', 'Health', 'Humanities', 'Mathematics', "
              "'Personal Culinary', 'Physical Science', 'Precision "
              "Production', 'Public Administration Social Service', "
              "'Resources', 'Science Technology', 'Security Law Enforcement', "
              "'Visual Performing'",
  'sat_avg': None,
  'sat_math': None,
  'sat_reading': None,
  'sat_writing': None,
  'student_size': 878,
  'tutio

In [11]:
tution_data = pd.DataFrame(dictionary_list)
print(f"No: of records:  {tution_data['College_Id'].count()}")
tution_data.head()

No: of records:  3200


,College_Id,price_calc_url,main_campus,branches,student_size,programs,act_writing,act_math,act_eng,act_net,...,sat_math,sat_avg,tution_instate,tution_out_of_state,net_cost_avg,net_cost_attend,net_cost_private,cost_inc_public,cost_inc_private,cost_inc_avg
0,194028,www.suny.edu/smarttrack/net-price-calculator/,1,1,929,"'Business Marketing', 'Computer', 'Family Cons...",NaN,NaN,NaN,NaN,...,NaN,NaN,6008.0,12638.0,10751.0,16374.0,NaN,15937.0,NaN,15937.0
1,194222,https://www.suny.edu/howmuch/netpricecalculato...,1,1,6190,"'Architecture', 'Business Marketing', 'Communi...",NaN,NaN,NaN,NaN,...,NaN,NaN,5484.0,10384.0,7452.0,14002.0,NaN,12709.0,NaN,12709.0
2,195322,sunysccc.edu/Admissions/Paying-for-SUNY-SCCC/F...,1,1,2677,"'Biological', 'Business Marketing', 'Computer'...",NaN,NaN,NaN,NaN,...,NaN,NaN,4944.0,9360.0,5780.0,11532.0,NaN,10778.0,NaN,10778.0
3,195988,sunysullivan.edu/financial-aid-tools-to-learn-...,1,1,996,"'Business Marketing', 'Communication', 'Comput...",NaN,NaN,NaN,NaN,...,NaN,NaN,5690.0,10504.0,8262.0,15222.0,NaN,13829.0,NaN,13829.0
4,196015,www.suny.edu/howmuch/?embed=n&cssUrl=www.canto...,1,1,3127,"'Business Marketing', 'Communication', 'Comput...",NaN,20.0,22.0,22.0,...,540.0,997.0,8389.0,12379.0,13728.0,22302.0,NaN,19009.0,NaN,19009.0


### Load the data into SQLite

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
engine = create_engine('sqlite:///Output/College_db.sqlite')
conn = engine.connect()
Base = declarative_base()

Base.metadata.create_all(engine)

In [19]:
college_data.to_sql('colleges', con=conn, if_exists='replace')

In [20]:
tution_data.to_sql('tutiondata', con=conn, if_exists='replace', index=False)

In [21]:
conn.close()